In [5]:
import pandas as pd

repo_name = "mall"

# Load the uploaded CSV file to inspect its structure
concept_data = pd.read_csv(f'../data/extracted_config_data/{repo_name}_concepts.csv')
file_data = pd.read_csv(f'../data/extracted_config_data/{repo_name}_files.csv')
option_data = pd.read_csv(f'../data/extracted_config_data/{repo_name}_options.csv')

# Load concept and file data and add changed internally property to the corresponding nodes

################################################################################################
# Create graph data for a given project
################################################################################################

print(len(option_data['File Path'].unique()))
print(len(file_data['File_path'].unique()))

40
41


In [9]:
import pandas as pd

repo_name = "mall"

# Load the uploaded CSV file to inspect its structure
concept_data = pd.read_csv(f'../data/extracted_config_data/{repo_name}_concepts.csv')
file_data = pd.read_csv(f'../data/extracted_config_data/{repo_name}_files.csv')
option_data = pd.read_csv(f'../data/extracted_config_data/{repo_name}_options.csv')

# Load concept and file data and add changed internally property to the corresponding nodes

################################################################################################
# Create graph data for a given project
################################################################################################

# check if concepts and files matches with concepts and files in option data
assert len(option_data['Concept'].unique()) == len(concept_data['Concept'])


artifacts_from_options = option_data['File Path'].unique()

# Create a list of unique nodes
concepts = concept_data[["Concept", "Change_count"]].apply(tuple, axis=1)
artifacts = file_data[["File_path", "Change_count"]].apply(tuple, axis=1)
options = option_data[['File Path', 'Option', 'Values', 'Changed internally', 'Changed globally', 'Removed']].apply(tuple, axis=1)

# Add technology nodes
nodes = [
    {
        'id': concept[0], 
        'changed_internally': concept[1],
        'type': 'concept'
    } for concept in concepts
]

# Add config file nodes
nodes += [
    {
        'id': artifact[0], 
        'changed_internally': artifact[1],
        'type': 'artifact'
    } for artifact in artifacts if artifact[0] in artifacts_from_options
]

# add option nodes
nodes += [
    {
        'id': f"{option[0]}:{option[1]}", 
        'values': str(option[2]), 
        'changed_internally': option[3], 
        'changed_globally': option[4], 
        'removed': option[5], 
        'type': 'option', 
    } for option in options
]

# Define links
links = []

# Link concepts to artifacts
for _, row in option_data.iterrows():
    links.append({'source': row['Concept'], 'target': row['File Path'], 'type': 'concept-artifact'})
    
# Link artifacts to options
for _, row in option_data.iterrows():
    artifact = row["File Path"]
    option = row["Option"]
    links.append({'source': row['File Path'], 'target': f"{artifact}:{option}", 'type': 'artifact-option'})


################################################################################################
# Add additional links to the graph data (Co-Changes of concepts)
################################################################################################
# Links concepts to concepts
concept_relations_path = f'../data/concept_cochanges/{repo_name}_concept_cochanges.csv'
concept_relations = pd.read_csv(concept_relations_path)

for _, row in concept_relations.iterrows():
    # Create a link for the concept relationship
    links.append({
        'source': row['Concept1'],
        'target': row['Concept2'],
        'type': 'concept-concept',
        'commit_window':  row['Commit Window'],
        'internal_weight': float(row['Percentage Internally']),
        'internal_count': row['Changed Internally'], 
        'across_projects': row["Across Projects"],
        'global_weight': float(row['Percentage globally']), 
        'global_count': row['Changed globally']  

    })

################################################################################################
# Add additional links to the graph data (Co-Changes of config file)
################################################################################################
# Links artifacts to artifacts
artifact_relations_path = f'../data/file_cochanges/{repo_name}_file_cochanges.csv'
artifact_relations = pd.read_csv(artifact_relations_path)

for _, row in artifact_relations.iterrows():
    # Create a link for the concept relationship
    links.append({
        'source': row['Artifact1'],
        'target': row['Artifact2'],
        'type': 'artifact-artifact',
        'commit_window': row['Commit Window'],
        'internal_weight': float(row['Percentage Internally']),
        'internal_count': row['Changed Internally'], 
        'across_projects': row["Across Projects"],
        'global_weight': float(row['Percentage globally']), 
        'global_count':  row['Changed globally']  
    })

################################################################################################
# Add additional links to the graph data (Co-Changes of config options)
################################################################################################
# Links options to options
option_relations_path = f'../data/option_cochanges/{repo_name}_option_cochanges.csv'
option_relations = pd.read_csv(option_relations_path)

for _, row in option_relations.iterrows():
    # Create a link for the concept relationship
    links.append({
        'source': f"{row['Artifact1']}:{row['Option1']}",
        'target': f"{row['Artifact2']}:{row['Option2']}",
        'source_option': row['Option1'].split(":")[-1],
        'target_option': row['Option2'].split(":")[-1],
        'type': 'option-option',
        'commit_window': row['Commit Window'],
        'internal_weight': float(row['Percentage Internally']),
        'internal_count': row['Changed Internally'], 
        #'across_projects': row["Across Projects"]
        #'global_weight': row['Percentage Globally'], 
        #'global_count': row['Changed Globally']  
    })


# Output the data
graph_data = {'nodes': nodes, 'links': links}

# Save the graph data as JSON for D3.js
import json
output_path = f'../data/graph_data/{repo_name}_graph_data.json'
with open(output_path, 'w') as f:
    json.dump(graph_data, f)

output_path

'../data/graph_data/mall_graph_data.json'